In [2]:
pip install caer canaro

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import caer
import canaro
import numpy as np
import cv2 as cv
import gc

In [4]:
IMG_SIZE = (80,80)
channels = 1
char_path = r'/kaggle/input/the-simpsons-characters-dataset/simpsons_dataset'

In [5]:
char_dict = {}
for char in os.listdir(char_path):
    char_dict[char] = len(os.listdir(os.path.join(char_path, char)))
    
# sort the dictionary in decending order
char_dict = caer.sort_dict(char_dict, descending = True)
char_dict

FileNotFoundError: [WinError 3] The system cannot find the path specified: '/kaggle/input/the-simpsons-characters-dataset/simpsons_dataset'

In [ ]:
characters = []
count = 0
for i in char_dict:
    characters.append(i[0])
    count += 1
    if count >= 10:
        break
characters

In [ ]:
# Create the training data

train = caer.preprocess_from_dir(char_path, characters, channels = channels, IMG_SIZE = IMG_SIZE, isShuffle= True)

In [ ]:
len(train)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(15,15))
plt.imshow(train[0][0], cmap = 'gray')
plt.show()

In [ ]:
featuresSet, labels = caer.sep_train(train, IMG_SIZE = IMG_SIZE)

In [ ]:
from tensorflow.keras.utils import to_categorical
# Normalize the featureSet ==> (0,1)
featureSet = caer.normalize(featuresSet)
labels = to_categorical(labels, len(characters))


In [ ]:
x_train, x_val, y_train, y_val = caer.train_val_split(featureSet, labels, val_ratio = 0.2)

In [ ]:
del train
del featureSet
del labels
gc.collect()

In [ ]:
BATCH_SIZE = 32
EPOCHS = 10

In [ ]:

# Image data generator

datagen = canaro.generators.imageDataGenerator()
train_gen = datagen.flow(x_train, y_train, batch_size = BATCH_SIZE)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten

In [ ]:
# Create a Sequential model
model = Sequential()

# Add convolutional layers
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], channels)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())

# Add dense layers
model.add(Dense(128, activation='relu'))
model.add(Dense(len(characters), activation='softmax'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [ ]:

model.summary()

In [ ]:
from tensorflow.keras.callbacks import LearningRateScheduler
callback_list = [LearningRateScheduler(canaro.lr_schedule)]

In [ ]:
training = model.fit(train_gen,
                    steps_per_epoch = len(x_train)//BATCH_SIZE,
                    epochs = EPOCHS,
                    validation_data = (x_val, y_val),
                    validation_steps = len(y_val)//BATCH_SIZE,
                     callbacks = callback_list)

In [ ]:
# Take one bart_simpsons image and try to predict
test_path = (r'/kaggle/input/the-simpsons-characters-dataset/kaggle_simpson_testset/kaggle_simpson_testset/charles_montgomery_burns_0.jpg')

img = cv.imread(test_path)

def prepare(img):
    img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    img = cv.resize(img, IMG_SIZE)
    img = caer.reshape(img, IMG_SIZE, 1)
    return img

In [ ]:
predictions = model.predict(prepare(img))


In [ ]:
print(characters[np.argmax(predictions[0])])